# Run quantum accurate geometry optimisation on a small molecule

This notebook shows how to geometry optimise a small molecule using RIMP2 accurate gradients.

In [ ]:
import os
from glob import glob
from pathlib import Path
import json

import rush

## 0) Setup

In [ ]:
# Define our project information
EXPERIMENT = "rush qm geo opt notebook"
MOLECULE_NAME = "cyclobutane"
TAGS = ["qdx", EXPERIMENT, MOLECULE_NAME]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=True)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

## 0.1) Initialize our rush client and fetch available module paths

In [ ]:
# By using the `build_provider_with_functions` method, we will also build
# helper functions calling each module
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

## 0.2) Get qm_geo_opt rush module

In [ ]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducible
# This will be done automatically if you use the `build_provider_with_functions`
# method
modules = await client.get_latest_module_paths()

In [ ]:
modules

{'qm_geo_opt': 'github:talo/qm_geo_opt/cef39be6050797d97ae3055ab0152d83d5e6c64b#qm_geo_opt_tengu',
 'gmx': 'github:talo/gmx_tengu_support/9cc36b5729e226a9e52ae7218d9e552fa16ad1fb#gmx_tengu',
 'dubai': 'github:talo/Dubai/a76eaabb0caa2196e390141cdac2e950ae29e665#dubai_tengu',
 'prepare_protein': 'github:talo/prepare_protein/83bed2ad1f01f495c94518717f9f5b1bd7fe855c#prepare_protein_tengu',
 'pdb2pqr': 'github:talo/prepare_protein/83bed2ad1f01f495c94518717f9f5b1bd7fe855c#pdb2pqr_tengu',
 'cairo': 'github:talo/cairo/665b83a4ed8ff5e811fcbd9e55ddf8f21e820aeb#cairo_tengu',
 'pbsa': 'github:talo/pbsa-cuda/399ed826b15af36391e7ab9cea93b5e551e784bf#pbsa_tengu',
 'qp_gen_inputs': 'github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#qp_gen_inputs',
 'qp_collate': 'github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#qp_collate',
 'pick_conformer': 'github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#pick_conformer',
 'pdbfixer': 'github:talo/tengu-prelude/e

In [ ]:
os.getenv("RUSH_URL")

'https://tengu.qdx.ai'

In [ ]:
help(client.qm_geo_opt)

Help on function qm_geo_opt in module rush.provider:

async qm_geo_opt(*args: [<class 'pathlib.Path'>, dict[str, ~T]], target: rush.graphql_client.enums.ModuleInstanceTarget | None = 'NIX_SSH_2', resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=0, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=10, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>]
    geometry optimise molecule with quantum accuracy
    
    Module version: `qm_geo_opt/cef39be6050797d97ae3055ab0152d83d5e6c64b`
    
    QDX Type Description:
    
        input: @Conformer;
        parameters: {
            basis_set: string,
            conv_threshold: f64,
            max_iterations: u32,
            use_internal_coords: bool
        }
        ->
        output: @(
            [Conformer],
            {


# 1) Run Geometry Optimisation

## 1.0) Create input geometry
We will be creating a qdxf conformer object from an unoptimised cyclobutane xyz

In [ ]:
def xyz_to_qdxf_topology(xyz_data: str):
    symbols = []
    geometry = []
    for line in xyz_data.splitlines()[2:]:
        symbol, x, y, z = line.split()
        symbols.append(symbol)
        geometry.extend([float(x), float(y), float(z)])
    return {"topology": {"symbols": symbols, "geometry": geometry}}


def qdxf_topology_to_xyz(qdxf_in: dict, name: str):
    symbols = qdxf_in["topology"]["symbols"]
    geometry = qdxf_in["topology"]["geometry"]
    xyz_data = f"{len(symbols)}\n{name}\n"
    for i in range(len(symbols)):
        symbol = symbols[i]
        x, y, z = geometry[3 * i : 3 * i + 3]
        xyz_data += f"  {symbol}"
        for coord in [x, y, z]:
            xyz_data += "     "
            if coord >= 0:
                xyz_data += " "
            xyz_data += f"{(coord):.4f}"
        xyz_data += "\n"
    return xyz_data

In [ ]:
cyclobutane_xyz = """12
cyclobutane
  C     -0.750      0.452     -0.417
  C     -0.696     -0.588      0.609
  C      0.820     -0.678      0.537
  C      0.892      0.417     -0.428
  H     -1.285      1.273      0.066
  H     -1.328      0.080     -1.263
  H     -1.225     -1.507      0.366
  H     -1.029     -0.162      1.555
  H      1.158     -1.594      0.054
  H      1.310     -0.477      1.488
  H      1.432      0.009     -1.290
  H      1.506      1.236     -0.056
"""

In [ ]:
cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

In [ ]:
# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
    json.dump(cyclobutane_qdxf, f)

## 1.1) Create geometry optimisation parameters

In [ ]:
params = {
    "use_internal_coords": True,
    "max_iterations": 100,
    "conv_threshold": 1e-4,
    "basis_set": "cc-pVDZ",
}

## 1.1) Call geometry optimisation over input geometry

In [ ]:
(geo_opt_out,) = await client.qm_geo_opt(
    cyclobutane_qdxf_input_file, params, target="NIX_SSH_3"
)
print(await geo_opt_out.get())

2024-02-01 13:15:37,882 - rush - INFO - Argument bc2fe0f0-5d89-46cd-953b-b09caa0d461a is now ModuleInstanceStatus.RESOLVING
2024-02-01 13:15:43,458 - rush - INFO - Argument bc2fe0f0-5d89-46cd-953b-b09caa0d461a is now ModuleInstanceStatus.ADMITTED
2024-02-01 13:15:56,891 - rush - INFO - Argument bc2fe0f0-5d89-46cd-953b-b09caa0d461a is now ModuleInstanceStatus.DISPATCHED
2024-02-01 13:16:02,448 - rush - INFO - Argument bc2fe0f0-5d89-46cd-953b-b09caa0d461a is now ModuleInstanceStatus.RUNNING
2024-02-01 13:16:24,716 - rush - INFO - Argument bc2fe0f0-5d89-46cd-953b-b09caa0d461a is now ModuleInstanceStatus.AWAITING_UPLOAD
[[{'topology': {'geometry': [-0.6777971, 0.24510759, -0.59337085, -0.70269954, -0.5826197, 0.71698254, 0.83522034, -0.7243974, 0.5833233, 0.8135363, 0.54939514, -0.29977712, -1.3783678, 1.0860906, -0.70158887, -0.7720476, -0.41545442, -1.4698635, -1.3166242, -1.4944503, 0.75519466, -0.9588622, 0.05847697, 1.5753573, 1.0906929, -1.6095256, -0.02059064, 1.447912, -0.7245677, 

## 1.2) Check calculation converged

In [ ]:
(topologies_out, results_out) = await geo_opt_out.get()
assert results_out["converged"]

# 2) Visualise pre-optimised and optimised geometries

In [ ]:
import py3Dmol

In [ ]:
pre_opt_view = py3Dmol.view()
pre_opt_view.addModel(cyclobutane_xyz, "xyz")
pre_opt_view.setStyle({"stick": {}})
pre_opt_view.zoomTo()
pre_opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
(topologies_out, _) = await geo_opt_out.get()
opt_cyclobutane_xyz = qdxf_topology_to_xyz(
    topologies_out[-1], "optimised cyclobutane"
)

opt_view = py3Dmol.view()
opt_view.addModel(opt_cyclobutane_xyz, "xyz", {})
opt_view.setStyle({"stick": {}})
opt_view.zoomTo()
opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol